In [268]:
import altair as alt
import numpy as np
import pandas as pd

In [269]:
fl = pd.read_csv("/Users/emma/Downloads/Finality Subsets/FL subsetfinalized.csv")
tx = pd.read_csv("/Users/emma/Downloads/Finality Subsets/TX subsetfinalized.csv")
wa = pd.read_csv("/Users/emma/Downloads/Finality Subsets/WA subsetfinalized.csv")

In [291]:
def dataset(df, year, state):
    """make dataset ready to run diff-diff and pre-post for opioids"""
    df["TRANSACTION_YEAR"] = df["TRANSACTION_YEAR"].astype("int")
    opioids_data = (df.groupby(["BUYER_STATE_x", "TRANSACTION_YEAR"], as_index=False).sum())
    opioids_data["policy"] = 0
    opioids_data.loc[opioids_data["TRANSACTION_YEAR"] > year, "policy"] = 1
    opioids_data["state"] = 0
    opioids_data.loc[opioids_data["BUYER_STATE_x"] == state, "state"] = 1
    df["death_rate"]= df["Deaths"] / df["population"]
    return opioids_data


In [293]:
FL = dataset(fl, 2010, "FL")

In [294]:
TX = dataset(tx, 2007, "TX")

In [295]:
WA = dataset(wa, 2012, "WA")

In [298]:
def pre_post(data, yvar, xvar, year,  alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    data1 = data.loc[(data["policy"]==0) & (data["state"] ==1), :]
    x = data1.loc[pd.notnull(data1[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions1 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before = smf.ols(f"{yvar} ~ {xvar}", data=data1).fit()
    model_predict = model_before.get_prediction(predictions1[xvar])
    predictions1[yvar] = model_predict.summary_frame()["mean"]
    predictions1[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg1 = alt.Chart(predictions1).mark_line().encode(x=xvar, y=yvar)
    ci1 = (
        alt.Chart(predictions1)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )
    
    data2 = data.loc[(data["policy"]==1) & (data["state"]==1),:]
    x = data2.loc[pd.notnull(data2[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions2 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after = smf.ols(f"{yvar} ~ {xvar}", data=data2).fit()
    model_predict_after = model_after.get_prediction(predictions2[xvar])
    predictions2[yvar] = model_predict_after.summary_frame()["mean"]
    predictions2[["ci_low", "ci_high"]] = model_predict_after.conf_int(alpha=alpha)

    # Build chart
    reg2 = alt.Chart(predictions2).mark_line().encode(x=xvar, y=yvar)
    ci2 = (
        alt.Chart(predictions2)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    overlay = pd.DataFrame({'x': [year]})
    vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
    
    chart = ci1 + ci2+ reg1 +reg2+ vline
    return predictions1, chart


In [299]:
def diff_diff(data, yvar, xvar, year, alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    data1 = data.loc[(data["policy"]==0) & (data["state"] == 0),:]
    x = data1.loc[pd.notnull(data1[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions1 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before = smf.ols(f"{yvar} ~ {xvar}", data=data1).fit()
    model_predict = model_before.get_prediction(predictions1[xvar])
    predictions1[yvar] = model_predict.summary_frame()["mean"]
    predictions1[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg1 = alt.Chart(predictions1).mark_line().encode(x=xvar, y=yvar)
    ci1 = (
        alt.Chart(predictions1)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    data11 = data.loc[(data["policy"]==0) & (data["state"] == 1),:]
    x = data11.loc[pd.notnull(data11[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions11 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before1 = smf.ols(f"{yvar} ~ {xvar}", data=data11).fit()
    model_predict1 = model_before1.get_prediction(predictions11[xvar])
    predictions11[yvar] = model_predict1.summary_frame()["mean"]
    predictions11[["ci_low", "ci_high"]] = model_predict1.conf_int(alpha=alpha)

    # Build chart
    reg11 = alt.Chart(predictions11).mark_line().encode(x=xvar, y=yvar)
    ci11 = (
        alt.Chart(predictions11)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )
    
    data2 = data.loc[(data["policy"]==1) & (data["state"] == 0),:]
    x = data2.loc[pd.notnull(data2[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions2 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after = smf.ols(f"{yvar} ~ {xvar}", data=data2).fit()
    model_predict_after = model_after.get_prediction(predictions2[xvar])
    predictions2[yvar] = model_predict_after.summary_frame()["mean"]
    predictions2[["ci_low", "ci_high"]] = model_predict_after.conf_int(alpha=alpha)

    # Build chart
    reg2 = alt.Chart(predictions2).mark_line().encode(x=xvar, y=yvar)
    ci2 = (
        alt.Chart(predictions2)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    data21 = data.loc[(data["policy"]==1) & (data["state"] == 1),:]
    x = data21.loc[pd.notnull(data21[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions21 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after1 = smf.ols(f"{yvar} ~ {xvar}", data=data21).fit()
    model_predict_after1 = model_after1.get_prediction(predictions21[xvar])
    predictions21[yvar] = model_predict_after1.summary_frame()["mean"]
    predictions21[["ci_low", "ci_high"]] = model_predict_after1.conf_int(alpha=alpha)

    # Build chart
    reg21 = alt.Chart(predictions21).mark_line().encode(x=xvar, y=yvar)
    ci21 = (
        alt.Chart(predictions21)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    overlay = pd.DataFrame({'x': [year]})
    vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
   
    chart = ci1 + ci2+ ci11+ ci21+reg1 +reg11 + reg2+ reg21+vline
    return chart


## Florida

> pre-post opioids

In [301]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
fit, chart = pre_post(FL, yvar, xvar, 2010, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff opioids

In [302]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
chart = diff_diff(FL, yvar, xvar, 2010)
chart

alt.LayerChart(...)

> pre-post death rate

In [313]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
fit, chart = pre_post(FL, yvar, xvar, 2010, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff for death rate

In [315]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
chart = diff_diff(FL, yvar, xvar, 2010, alpha=0.05)
chart

alt.LayerChart(...)

## Washington

> pre-post opioids

In [303]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
fit, chart = pre_post(WA, yvar, xvar, 2012, alpha=0.05)
chart

/Users/emma/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

> diff-diff opioids

In [304]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
chart = diff_diff(WA, yvar, xvar, 2012)
chart

/Users/emma/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

> pre-post for death rate

In [316]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
fit, chart = pre_post(WA, yvar, xvar, 2012, alpha=0.05)
chart

/Users/emma/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

> diff-diff for death rate

In [317]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
chart = diff_diff(WA, yvar, xvar, 2012)
chart

/Users/emma/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

In [321]:
TX.sample(10)

,BUYER_STATE_x,TRANSACTION_YEAR,Opioids_Shipment_IN_GM,fips,population,Deaths,opioid_shipment_population_ratio,policy,state,death_rate
2,CA,2008,9.178293e+06,1.145100e+10,5.750294e+12,491087889.0,22.478334,1,0,0.000085
63,TX,2006,3.995719e+06,6.763874e+10,1.650067e+12,154715467.0,34.782531,0,1,0.000094
46,ND,2007,9.777242e+04,1.414860e+09,2.153756e+09,0.0,5.842829,0,0,0.000000
40,MD,2010,2.647307e+06,1.127227e+10,2.272560e+11,23478221.0,13.200959,1,0,0.000103
74,UT,2008,1.082797e+06,1.316611e+10,1.438266e+11,26804902.0,11.128598,1,0,0.000186
66,TX,2009,5.123285e+06,8.487088e+10,2.125745e+12,209301964.0,44.913991,1,1,0.000098
12,CO,2009,1.454814e+06,3.372206e+09,1.457196e+11,20486536.0,15.105026,1,0,0.000141
20,GA,2008,2.587313e+06,1.128102e+10,2.200241e+11,17652269.0,44.744939,1,0,0.000080
17,CO,2014,1.709159e+06,3.500307e+09,1.682682e+11,28281644.0,18.020627,1,0,0.000168
76,UT,2010,1.203765e+06,1.484382e+10,1.605500e+11,25255225.0,12.941672,1,0,0.000157


## Texas

In [322]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
fit, chart = pre_post(TX, yvar, xvar, 2007, alpha=0.05)
chart

/Users/emma/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

In [323]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
chart = diff_diff(TX, yvar, xvar, 2007, alpha=0.05)
chart

/Users/emma/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)